# Exemplo base carla simulator com yolo

* Aqui temos um exemplo base e aplicação de yolo no carla simulator

In [1]:
# All imports
import carla
import time
import cv2
import numpy as np
import random
from ultralytics import YOLO

In [2]:
import sys
sys.path.append('C:\\Users\\eglm\\Documents\\CARLA_0.9.14\\PythonAPI\\carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [3]:
# Connect to the carla

client = carla.Client('localhost', 2000)

In [4]:
# World
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

In [5]:
# ambiente
weather = carla.WeatherParameters(
        cloudiness=30,
        precipitation=0,
        sun_altitude_angle=90,
        precipitation_deposits =0,
        fog_density =0.9,
        wetness = 75)
world.set_weather(weather)

# trafego
# traffic_manager = client.get_trafficmanager(8000)
blueprints = world.get_blueprint_library().filter('*vehicle*')

# clean up
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

# ensure sync mode on 
# settings = world.get_settings()
# settings.synchronous_mode = True
# settings.fixed_delta_seconds = 0.1
# settings.no_rendering_mode = False
# traffic_manager.set_synchronous_mode(True)
# traffic_manager.global_percentage_speed_difference(70)

# world.apply_settings(settings)
    
# generate traffic limited number
car_numbers = 30
for n, transform in enumerate(spawn_points):
    if n > car_numbers: # good volume of traffic - up to 1/3 of available spawn points
        break
    blueprint = random.choice(blueprints)
    traffic_vehicle = world.try_spawn_actor(blueprint, transform)
    if traffic_vehicle != None:# spawn the cars and set their autopilot and light state all together
        traffic_vehicle.set_light_state(carla.VehicleLightState.LowBeam)
        traffic_vehicle.set_autopilot(True)


In [6]:
# look for a blueprint of mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

# spanw a car
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0],start_point)

# setup camera

CAMERA_POS_Z = 1.6
CAMERA_POS_X = 0.9

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x','640')
camera_bp.set_attribute('image_size_y','360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x = CAMERA_POS_X))
camera = world.spawn_actor(camera_bp,camera_init_trans, attach_to= vehicle)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image':np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

In [7]:
point_a = start_point.location

sampling_resolution = 1
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)

# get de longest possible route
distance = 0
for loc in spawn_points:

    cur_route = grp.trace_route(point_a,loc.location)
    if len(cur_route)>distance:
        distance = len(cur_route)
        route = cur_route

for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
        color = carla.Color(r=0,g=255,b=0), life_time = 60.0,
        persistent_lines=True)

In [8]:
from ultralytics import YOLO 

model = YOLO("yolov8n.pt") 

In [9]:
def getObjetcs(model, image):
    results = model.predict(image, conf=0.7)  
    objectsYOLO = {}
    objectYOLO = {
        "classId": 0,
        "cords": [0, 0, 0, 0],
        "conf": 0,
    }
    #comentar caso não queira debugar
    imageYOLO = image
    if(results[0]):
        result = results[0]
        objectYOLOID = 0
        #comentar caso não queira debugar
        imageYOLO = result.plot() 
        for box in result.boxes:
            objectYOLO["classId"]  = result.names[box.cls[0].item()]
            objectYOLO["coords"] = box.xyxy[0].tolist()
            objectYOLO["coords"] = [round(x) for x in objectYOLO["coords"]]
            objectYOLO["conf"] = round(box.conf[0].item(), 2)
            objectYOLOID = objectYOLOID + 1
            objectsYOLO[str(objectYOLOID)] = objectYOLO
            #comentar caso não queira debugar
            #print("Objeto Yolo ({}):\n-classId: {}\n-coords: {}\n-conf: {}\n-depth: {}\n\n".format(object, objectsYOLO[str(objectYOLOID)]['classId'], objectsYOLO[str(objectYOLOID)]['coords'], objectsYOLO[str(objectYOLOID)]['conf'], objectsYOLO[str(objectYOLOID)]['distance']))
    return str(objectsYOLO), imageYOLO

In [10]:
for way_point in route:
    vehicle.set_transform(way_point[0].transform)

    frame = camera_data['image']            
    string_yolo, imageYOLO = getObjetcs(model= model, image = frame[:, :, :3])

    cv2.imshow('fake self-driving', imageYOLO)
    cv2.waitKey(1)

time.sleep(2)
cv2.destroyAllWindows()
camera.stop()

# utility script of destruction
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()


0: 384x640 1 car, 622.6ms
Speed: 117.7ms preprocess, 622.6ms inference, 114.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 106.7ms
Speed: 4.0ms preprocess, 106.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 104.7ms
Speed: 5.0ms preprocess, 104.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 104.7ms
Speed: 2.0ms preprocess, 104.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 109.7ms
Speed: 4.0ms preprocess, 109.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 121.7ms
Speed: 4.0ms preprocess, 121.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 111.2ms
Speed: 4.0ms preprocess, 111.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 123.7ms
Speed: 3.0ms preprocess,

In [39]:
# utility script of destruction
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()